In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
league_fp = os.path.join('data', '2022_LoL_esports_match_data_from_OraclesElixir.csv')
league_raw = pd.read_csv(league_fp)

/Users/elim-mbp-01/.pyenv/versions/3.8.16/envs/dsc80_39/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Which role “carries” (does the best) in their team more often: ADCs (Bot lanes) or Mid laners?

# Is Talon (tutor Costin’s favorite champion) more likely to win or lose any given match?


In [4]:
matchgb = league_raw.groupby("gameid").get_group("ESPORTSTMNT01_2690210")
matchgb_columns = matchgb.columns.to_list()
matchgb_columns


['gameid',
 'datacompleteness',
 'url',
 'league',
 'year',
 'split',
 'playoffs',
 'date',
 'game',
 'patch',
 'participantid',
 'side',
 'position',
 'playername',
 'playerid',
 'teamname',
 'teamid',
 'champion',
 'ban1',
 'ban2',
 'ban3',
 'ban4',
 'ban5',
 'gamelength',
 'result',
 'kills',
 'deaths',
 'assists',
 'teamkills',
 'teamdeaths',
 'doublekills',
 'triplekills',
 'quadrakills',
 'pentakills',
 'firstblood',
 'firstbloodkill',
 'firstbloodassist',
 'firstbloodvictim',
 'team kpm',
 'ckpm',
 'firstdragon',
 'dragons',
 'opp_dragons',
 'elementaldrakes',
 'opp_elementaldrakes',
 'infernals',
 'mountains',
 'clouds',
 'oceans',
 'chemtechs',
 'hextechs',
 'dragons (type unknown)',
 'elders',
 'opp_elders',
 'firstherald',
 'heralds',
 'opp_heralds',
 'firstbaron',
 'barons',
 'opp_barons',
 'firsttower',
 'towers',
 'opp_towers',
 'firstmidtower',
 'firsttothreetowers',
 'turretplates',
 'opp_turretplates',
 'inhibitors',
 'opp_inhibitors',
 'damagetochampions',
 'dpm',
 'd

In [7]:
league_raw.columns.to_list()

['gameid',
 'datacompleteness',
 'url',
 'league',
 'year',
 'split',
 'playoffs',
 'date',
 'game',
 'patch',
 'participantid',
 'side',
 'position',
 'playername',
 'playerid',
 'teamname',
 'teamid',
 'champion',
 'ban1',
 'ban2',
 'ban3',
 'ban4',
 'ban5',
 'gamelength',
 'result',
 'kills',
 'deaths',
 'assists',
 'teamkills',
 'teamdeaths',
 'doublekills',
 'triplekills',
 'quadrakills',
 'pentakills',
 'firstblood',
 'firstbloodkill',
 'firstbloodassist',
 'firstbloodvictim',
 'team kpm',
 'ckpm',
 'firstdragon',
 'dragons',
 'opp_dragons',
 'elementaldrakes',
 'opp_elementaldrakes',
 'infernals',
 'mountains',
 'clouds',
 'oceans',
 'chemtechs',
 'hextechs',
 'dragons (type unknown)',
 'elders',
 'opp_elders',
 'firstherald',
 'heralds',
 'opp_heralds',
 'firstbaron',
 'barons',
 'opp_barons',
 'firsttower',
 'towers',
 'opp_towers',
 'firstmidtower',
 'firsttothreetowers',
 'turretplates',
 'opp_turretplates',
 'inhibitors',
 'opp_inhibitors',
 'damagetochampions',
 'dpm',
 'd

In [21]:
missing_values = league_raw.isna().sum(axis=0)
missing_values[missing_values > 0].to_frame().to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>url</th>\n      <td>126984</td>\n    </tr>\n    <tr>\n      <th>split</th>\n      <td>41316</td>\n    </tr>\n    <tr>\n      <th>patch</th>\n      <td>108</td>\n    </tr>\n    <tr>\n      <th>playername</th>\n      <td>24900</td>\n    </tr>\n    <tr>\n      <th>playerid</th>\n      <td>27069</td>\n    </tr>\n    <tr>\n      <th>teamname</th>\n      <td>54</td>\n    </tr>\n    <tr>\n      <th>teamid</th>\n      <td>2124</td>\n    </tr>\n    <tr>\n      <th>champion</th>\n      <td>24900</td>\n    </tr>\n    <tr>\n      <th>ban1</th>\n      <td>2334</td>\n    </tr>\n    <tr>\n      <th>ban2</th>\n      <td>2202</td>\n    </tr>\n    <tr>\n      <th>ban3</th>\n      <td>2520</td>\n    </tr>\n    <tr>\n      <th>ban4</th>\n      <td>2388</td>\n    </tr>\n    <tr>\n      <th>ban5</th>\n      <td>2712</td>\n    </tr>\n 

In [22]:
league_test = league_raw.copy()
league_test["ban1_missing"] = league_test["ban1"].isna()

In [39]:
def tvd(dist1: pd.Series, dist2: pd.Series):
    dist1 = dist1 / dist1.sum() # normalize
    dist2 = dist2 / dist2.sum()
    return 0.5 * np.sum(np.abs(dist1 - dist2))

def tvd2(df: pd.DataFrame, gb_col, col: str):
    df.groupby("col")

In [47]:
lt_gb = league_test.groupby("ban1_missing")["ban2"]
lt_gb_vc = lt_gb.value_counts()

tvd(lt_gb_vc[True], lt_gb_vc[False])

0.35992082925236696

In [50]:
def permutation_test(df: pd.DataFrame, col_tf: str, col_i: str):
    df = df.copy()
    
    test_stats = []
    for _ in range(100):
        df[col_tf] = np.random.permutation(df[col_tf])
        df_gb = df.groupby(col_tf)[col_i].value_counts()
        test_stats.append(tvd(df_gb[True], df_gb[False]))
        
    return np.array(test_stats)

permutation_test(league_test, "ban1_missing", "ban2")

array([0.07166055, 0.07830172, 0.08730381, 0.08058253, 0.07845274,
       0.07094326, 0.07597582, 0.0761883 , 0.08412171, 0.07447078,
       0.06740802, 0.08414119, 0.07866214, 0.07554668, 0.08097894,
       0.06906338, 0.07459523, 0.07552102, 0.07596512, 0.07549603,
       0.07568128, 0.06839647, 0.08947518, 0.08900352, 0.08024738,
       0.0802104 , 0.07755819, 0.07236317, 0.07582941, 0.08189889,
       0.06303442, 0.0898507 , 0.06947786, 0.0788992 , 0.07539841,
       0.06954632, 0.08442251, 0.07816316, 0.08247678, 0.070596  ,
       0.07791224, 0.08571708, 0.07361935, 0.07037079, 0.08733109,
       0.07475401, 0.0784057 , 0.08031053, 0.07740424, 0.08681113,
       0.07659932, 0.07558076, 0.06829669, 0.0827546 , 0.07069092,
       0.07976446, 0.07989671, 0.07275217, 0.09719434, 0.06733284,
       0.07076266, 0.07250638, 0.0736366 , 0.07832212, 0.08297503,
       0.06576771, 0.08352588, 0.07922453, 0.07365809, 0.07697773,
       0.06300961, 0.08256421, 0.07495533, 0.0742709 , 0.07742

In [32]:
league_test.loc[
    league_test["ban1_missing"] == True,
    "ban2"
].value_counts()

Draven          18
Aatrox          12
Renata Glasc    12
Azir            12
Senna            6
Ahri             6
Aphelios         6
Riven            6
Sejuani          6
Amumu            6
Akali            6
Vayne            6
Taliyah          6
Ornn             6
Caitlyn          6
Teemo            6
Renekton         6
Volibear         6
Ezreal           6
Lillia           6
Poppy            6
Syndra           6
Hecarim          6
Yone             6
Name: ban2, dtype: int64

In [25]:
league_test.groupby("ban1_missing")["ban2"].mean()

TypeError: can only concatenate str (not "int") to str

In [11]:

faker_data = league_raw.loc[
    # (league_raw["champion"] == "Talon"),
    (league_raw["playername"] == "Faker"),
    ["teamname", "playername", "champion", "result"]
]
faker_data["result"].mean()
# Confirmed that 1 means win, 0 means loss
# https://gol.gg/players/player-stats/48/season-S12/split-ALL/tournament-ALL/J (74.3% WR)

0.7430555555555556

In [16]:
talon_data = league_raw.loc[
    (league_raw["champion"] == "Talon"),
    ["teamname", "playername", "champion", "result"]
]
talon_data.shape, talon_data["result"].mean()

((65, 4), 0.5538461538461539)

In [ ]:
# EGR - early game rating
# MLR - mid late game rating

In [28]:
matchgb[["participantid", "teamname", "playername", "champion"]]

,participantid,teamname,playername,champion
0,1,Fredit BRION Challengers,Soboro,Renekton
1,2,Fredit BRION Challengers,Raptor,Xin Zhao
2,3,Fredit BRION Challengers,Feisty,LeBlanc
3,4,Fredit BRION Challengers,Gamin,Samira
4,5,Fredit BRION Challengers,Loopy,Leona
5,6,Nongshim RedForce Challengers,DnDn,Gragas
6,7,Nongshim RedForce Challengers,Sylvie,Viego
7,8,Nongshim RedForce Challengers,FIESTA,Viktor
8,9,Nongshim RedForce Challengers,vital,Jinx
9,10,Nongshim RedForce Challengers,Blessing,Alistar


In [ ]:
league_raw.